In [1]:
import torch
import pickle
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from tabpfn import TabPFNClassifier  
from functools import partial
import tabpfn.encoders as encoders
from tabpfn.scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, load_model_workflow_my
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn  # 确保引入 nn 模块
import torch.optim as optim  # 确保引入 optim 模块

In [2]:
# 加载数据
train = pd.read_csv('/root/autodl-fs/data/train_revise+45缩减到100特征 数量1000个 去掉三列和Name.csv')  

# 分离特征和标签
X = train.drop(['senolytic'], axis=1).values
y = train['senolytic'].values

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用 SMOTE 对训练集进行过采样
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# 确保数据加载时的目标标签为 Long 类型
X_train_tensor = torch.tensor(X_train_balanced, dtype=torch.float32).cuda()
y_train_tensor = torch.tensor(y_train_balanced, dtype=torch.long).cuda()  # 确保为 long 类型

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
y_test_tensor = torch.tensor(y_test, dtype=torch.long).cuda()  # 确保为 long 类型


# 构造 DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
# 加载 TabPFNClassifier 模型
model = TabPFNClassifier(device='cuda', model_file="/root/autodl-fs/Sq-TabPFN/tabpfn/smote+sq_models_diff/prior_diff_real_checkpoint_n_1_epoch_110.cpkt")
transformer_model = model.model[2]  # 获取模型的 transformer 模块

# 加载保存的模型状态
checkpoint_path = "/root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调解码器模型/prior_diff_real_checkpoint_n_1_epoch_1.cpkt"
checkpoint = torch.load(checkpoint_path)

# 加载模型权重
transformer_model.load_state_dict(checkpoint['model_state_dict'])
transformer_model.cuda()

Loading /root/autodl-fs/Sq-TabPFN/tabpfn/smote+sq_models_diff/prior_diff_real_checkpoint_n_1_epoch_110.cpkt
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


TransformerModel(
  (transformer_encoder): TransformerEncoderDiffInit(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (encoder): Linear(in_features=100, out_features=512, bias=True)
  (y_encoder): Linear(in_features=1, out_features=512, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): GELU(approximate='none')

In [4]:
# 冻结解码器参数
for name, param in transformer_model.decoder.named_parameters():
    param.requires_grad = False

In [5]:

# 设置优化器和损失函数
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, transformer_model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


'''
# 设置优化器和损失函数
criterion = nn.BCEWithLogitsLoss()  # 二分类问题的损失函数
optimizer = optim.Adam(filter(lambda p: p.requires_grad, transformer_model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
'''


'\n# 设置优化器和损失函数\ncriterion = nn.BCEWithLogitsLoss()  # 二分类问题的损失函数\noptimizer = optim.Adam(filter(lambda p: p.requires_grad, transformer_model.parameters()), lr=1e-4)\nscheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)\n'

In [6]:
# 解冻 encoder 的所有参数
for name, param in transformer_model.encoder.named_parameters():
    param.requires_grad = True

# 获取 transformer_encoder 中的层数
num_encoder_layers = len(transformer_model.transformer_encoder.layers)

# 设置保存路径
save_dir = "/root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器模型"
best_val_loss = float('inf')  # 用于记录最佳验证损失

# 逐层解冻 transformer_encoder
for i in range(num_encoder_layers):
    print(f"Unfreezing encoder layer {i+1}/{num_encoder_layers}")
    for name, param in transformer_model.transformer_encoder.layers[i].named_parameters():
        param.requires_grad = True

    single_eval_pos = 0  # 设置默认评估位置

    # 在训练循环中
    for epoch in range(5):  # 每层训练 5 个 epoch
        transformer_model.train()
        total_loss = 0.0

        # 训练阶段
        for batch in train_loader:
            x, y = batch
            optimizer.zero_grad()
            outputs = transformer_model((x, y.float()), single_eval_pos=single_eval_pos).squeeze(1)
            loss = criterion(outputs, y.long())  # 修正为 long 类型
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # 验证阶段
        transformer_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in test_loader:
                x, y = batch
                outputs = transformer_model((x, y.float()), single_eval_pos=single_eval_pos).squeeze(1)
                loss = criterion(outputs, y.long())
                val_loss += loss.item()

        avg_val_loss = val_loss / len(test_loader)

        # 打印训练和验证损失
        print(f"Layer {i+1}, Epoch {epoch+1}/5 - Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # 保存验证集损失最小的模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_path = f"{save_dir}/prior_diff_real_checkpoint_n_{i+1}_epoch_{epoch+1}.cpkt"
            torch.save({
                'model_state_dict': transformer_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, save_path)
            print(f"New best model saved at epoch {epoch+1} with Validation Loss: {avg_val_loss:.4f} to {save_path}")

Unfreezing encoder layer 1/12
Layer 1, Epoch 1/5 - Train Loss: 0.7380, Validation Loss: 0.6321
New best model saved at epoch 1 with Validation Loss: 0.6321 to /root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器模型/prior_diff_real_checkpoint_n_1_epoch_1.cpkt
Layer 1, Epoch 2/5 - Train Loss: 0.6411, Validation Loss: 0.5463
New best model saved at epoch 2 with Validation Loss: 0.5463 to /root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器模型/prior_diff_real_checkpoint_n_1_epoch_2.cpkt
Layer 1, Epoch 3/5 - Train Loss: 0.6305, Validation Loss: 0.7773
Layer 1, Epoch 4/5 - Train Loss: 0.6404, Validation Loss: 0.7169
Layer 1, Epoch 5/5 - Train Loss: 0.6099, Validation Loss: 0.8282
Unfreezing encoder layer 2/12
Layer 2, Epoch 1/5 - Train Loss: 0.6595, Validation Loss: 0.7568
Layer 2, Epoch 2/5 - Train Loss: 0.5862, Validation Loss: 0.4234
New best model saved at epoch 2 with Validation Loss: 0.4234 to /root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器模型/prior_diff_real_checkpoint_n_2_epoch_2.cpkt
Layer 2, Epoch 3/

In [8]:
# 加载模型权重
checkpoint_path = "/root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器模型/prior_diff_real_checkpoint_n_9_epoch_2.cpkt"
checkpoint = torch.load(checkpoint_path)

# 假设 transformer_model 是已经初始化的 TransformerModel
model = transformer_model  # 使用同一个模型类实例
model.load_state_dict(checkpoint['model_state_dict'])  # 加载权重
model.cuda()  # 将模型移动到 GPU 上

TransformerModel(
  (transformer_encoder): TransformerEncoderDiffInit(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (encoder): Linear(in_features=100, out_features=512, bias=True)
  (y_encoder): Linear(in_features=1, out_features=512, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): GELU(approximate='none')

In [9]:
# 模型验证和指标计算
def evaluate_model(model, test_loader):
    model.eval()  # 切换到评估模式
    y_true = []
    y_pred = []
    y_prob = []  # 用于 ROC 和 PR 曲线

    with torch.no_grad():  # 禁用梯度计算
        for batch in test_loader:
            x, y = batch
            src = (x, y.float())  # 构造符合模型输入格式的元组
            outputs = model(src, single_eval_pos=0)
            
            # 获取预测类别和概率
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_prob.extend(outputs[:, 1].cpu().numpy())  # 假设二分类，取概率值

    # 转换为 NumPy 数组
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)

    # 计算指标
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    precision_vals, recall_vals, _ = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall_vals, precision_vals)

    # 打印指标
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")

    return y_true, y_pred, y_prob, precision_vals, recall_vals, roc_auc, pr_auc

# 评估模型性能
y_true, y_pred, y_prob, precision_vals, recall_vals, roc_auc, pr_auc = evaluate_model(transformer_model, test_loader)

Accuracy: 0.8350
Precision: 0.5000
Recall: 0.6667
F1 Score: 0.5714
ROC AUC: 0.8100
PR AUC: 0.5297
